In [ ]:
import numpy as np
import pandas as pd
from model_tuning import tune,  conformal_predict
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor



df_model = pd.read_csv("../data/hdb_final_dataset.csv")


In [2]:
df_model.describe()

,floor_area_sqm,lease_commence_date,remaining_lease,adjusted_resale_price,adj_resale_price_per_sqm,latitude,longitude,min_dist_sch,min_dist_cbd,min_dist_mrt,storey_median
count,238802.000000,238802.000000,238802.000000,2.388020e+05,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000
mean,96.961016,1995.410290,73.969732,5.616875e+05,5849.807495,1.367591,103.840889,1.775018,12.068064,0.909946,8.665422
std,24.055285,13.850938,13.649046,1.865108e+05,1521.092931,0.042838,0.071427,1.311574,4.359247,0.781477,5.852919
min,31.000000,1966.000000,40.000000,1.568250e+05,2340.671642,1.270380,103.685228,0.049585,0.721959,0.014630,2.000000
25%,81.000000,1985.000000,63.000000,4.248649e+05,4844.023989,1.337020,103.778110,0.856899,9.291233,0.431099,5.000000
50%,93.000000,1996.000000,74.000000,5.279775e+05,5516.060518,1.366806,103.846014,1.478137,12.983033,0.694491,8.000000
75%,112.000000,2006.000000,86.000000,6.587971e+05,6447.773656,1.395723,103.898879,2.268127,15.129038,1.088400,11.000000
max,366.700000,2021.000000,97.000000,1.619760e+06,16017.509677,1.457071,103.987805,7.581888,19.757794,4.837718,50.000000


In [2]:
from sklearn.model_selection import train_test_split
exog = ['remaining_lease', 'min_dist_sch', 'storey_median', 'min_dist_mrt',
        'floor_area_sqm', 'min_dist_cbd', 'flat_type_1 ROOM', 'flat_type_2 ROOM', 
        'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE', 
        'flat_type_MULTI-GENERATION']
 
X = df_model[exog]
y = df_model['adjusted_resale_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#%pip install xgboost

In [9]:
"""1. get a bse set of params for the train set
    use randomsearch to iterate thru the possible params"""

xgb_model = xgb.XGBRegressor(objective = "reg:squarederror") ## benchmark model 

bst_final=tune(xgb_model, X_train,y_train,
                                   X_test,y_test,
                                   model_type="xgboost",
                                   n_iter=20, 
                                   cv=5, 
                                   verbose=1, 
                                   n_jobs=-1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:44:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:44:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:44:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:44:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytia

[0]	eval-rmse:475715.98182
[1]	eval-rmse:383391.60599
[2]	eval-rmse:309857.57752
[3]	eval-rmse:251818.73823


/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:52:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[4]	eval-rmse:204991.31516
[5]	eval-rmse:168236.52899
[6]	eval-rmse:138991.39375
[7]	eval-rmse:116034.00310
[8]	eval-rmse:97908.40082
[9]	eval-rmse:83281.80980
[10]	eval-rmse:71869.03138
[11]	eval-rmse:63507.88375
[12]	eval-rmse:57350.23736
[13]	eval-rmse:52944.33450
[14]	eval-rmse:49781.33525
[15]	eval-rmse:47216.71247
[16]	eval-rmse:45153.39058
[17]	eval-rmse:43850.40601
[18]	eval-rmse:42969.81847
[19]	eval-rmse:42254.31634
[20]	eval-rmse:41663.05329
[21]	eval-rmse:41414.50636
[22]	eval-rmse:41294.46389
[23]	eval-rmse:41082.58777
[24]	eval-rmse:41155.75334
[25]	eval-rmse:40833.86498
[26]	eval-rmse:40754.19666
[27]	eval-rmse:40493.06220
[28]	eval-rmse:40208.06680
[29]	eval-rmse:40193.81013
[30]	eval-rmse:40026.56179
[31]	eval-rmse:39996.51578
[32]	eval-rmse:39836.36363
[33]	eval-rmse:39434.77171
[34]	eval-rmse:39135.51511
[35]	eval-rmse:39046.81438
[36]	eval-rmse:38858.24687
[37]	eval-rmse:38702.50209
[38]	eval-rmse:38457.70611
[39]	eval-rmse:38371.48339
[40]	eval-rmse:38255.23328
[41

In [4]:
## compare with randomforrest regressors

rf=RandomForestRegressor(random_state=42)
random_forest=tune(rf, X_train,y_train,
                                   X_test,y_test,
                                   model_type="random_forest",
                                   n_iter=20, 
                                   cv=5, 
                                   verbose=1, 
                                   n_jobs=-1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/fytian/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


## After hypertuning the two model, we compare the performance of RF vs XGBOOOST using RMSE, MAPE and %overprediction

In [10]:
from sklearn.metrics import r2_score, mean_squared_error
y_pred=random_forest.predict(X_test)
y_insample_pred=random_forest.predict(X_train)
tolerance = 0.05 * y_test
underprice_pct = ((y_test - y_pred) > tolerance).mean() 
print(f"% Underpricing: {underprice_pct * 100:.2f}%")

overprice_pct = ((y_pred - y_test) > tolerance).mean() 
print(f"% Overpricing: {overprice_pct * 100:.2f}%")


def mape (y_actual, y_predicted): 
    mape = np.mean(np.abs((y_actual - y_predicted)/y_actual))*100 
    return mape 

metrics = {
    "Metric": ["R² Score", "RMSE", "MAPE"],
    "RF Train": [round(r2_score(y_train, y_insample_pred), 3),
                  round(np.sqrt(mean_squared_error(y_train, y_insample_pred)), 4),
                  round(mape(y_train, y_insample_pred), 4)],
    "RF Test": [round(r2_score(y_test, y_pred), 3),
                 round(np.sqrt(mean_squared_error(y_test, y_pred)), 4),
                 round(mape(y_test, y_pred), 4)]
}

model_evaluation_df = pd.DataFrame(metrics)
print(model_evaluation_df)


% Underpricing: 17.36%
% Overpricing: 20.13%
     Metric    RF Train     RF Test
0  R² Score      0.9870      0.9610
1      RMSE  21442.4299  36645.7040
2      MAPE      2.8940      4.8657


In [11]:
from sklearn.metrics import r2_score, mean_squared_error
pred_xg_train = bst_final.predict(xgb.DMatrix(X_train)) 
pred_xg_test = bst_final.predict(xgb.DMatrix(X_test))  

# diff = y_test - pred_xg_test
# underprice_pct = (diff > 10000).mean()  # Only count when underpriced by more than 10k (30%)

# we only count it as underpricing if it is more than 5% below actual (14%)
tolerance = 0.05 * y_test
underprice_pct = ((y_test - pred_xg_test) > tolerance).mean() 
print(f"% Underpricing: {underprice_pct * 100:.2f}%")

overprice_pct = ((pred_xg_test - y_test) > tolerance).mean() 
print(f"% Overpricing: {overprice_pct * 100:.2f}%")

def mape (y_actual, y_predicted): 
    mape = np.mean(np.abs((y_actual - y_predicted)/y_actual))*100 
    return mape 

metrics = {
    "Metric": ["R² Score", "RMSE", "MAPE"],
    "XGB Train": [round(r2_score(y_train, pred_xg_train), 3),
                  round(np.sqrt(mean_squared_error(y_train, pred_xg_train)), 4),
                  round(mape(y_train, pred_xg_train), 4)],
    "XGB Test": [round(r2_score(y_test, pred_xg_test), 3),
                 round(np.sqrt(mean_squared_error(y_test, pred_xg_test)), 4),
                 round(mape(y_test, pred_xg_test), 4)]
}

model_evaluation_df = pd.DataFrame(metrics)
print(model_evaluation_df)

% Underpricing: 13.03%
% Overpricing: 21.89%
     Metric   XGB Train    XGB Test
0  R² Score      0.9940      0.9660
1      RMSE  13843.5232  34500.4440
2      MAPE      1.8377      4.5768


## We decided on using XGBOOST which is traiend using assymetric loss , such that it favor over prediction, and it have lower RMSE, We now perform conformal prediction on its predicted output.

In [13]:
"""3. This is the final step, our model predict the value + the conformal interval of alpha%, alpha can be adjusted"""


y_pred = bst_final.predict(xgb.DMatrix(X_test)) 

# STEP 3: Calculate residuals and quantile threshold q
residuals = np.abs(y_test - y_pred)

alpha = 0.1  # for 90% confidence
q = np.quantile(residuals, 1 - alpha) 

## this so FE can change alpha top get a diff  q

In [7]:
## save the model for front end use 


import joblib

model_package = {
    'model': bst_final,
    'q_value': q
}

joblib.dump(model_package, '/Users/fytian/Desktop/year 3/sem 2/DSE3101/hdbproject/src/final_model.pkl')

['/Users/fytian/Desktop/year 3/sem 2/DSE3101/hdbproject/src/final_model.pkl']

In [14]:
"""This fucntion takes in our best model bst_final. and input from FE  and the q defined above
output the predicted price and the conformal_prediction of 90% interval
exog = ['remaining_lease', 'min_dist_sch', 'storey_median', 'min_dist_mrt',
        'floor_area_sqm', 'min_dist_cbd', 

        ##the below are 0 or 1 
        'flat_type_1 ROOM', 'flat_type_2 ROOM', 
        'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE', 
        'flat_type_MULTI-GENERATION' ]
"""

##important function for FE to use 
conformal_predict(bst_final,[56,0.63,4,0.54,145,10.16,0,0,1,0,0,0,0],q)

 Predicted: 1002331.75, 90% CI: [948780.12, 1055883.38]


(array([1002331.75], dtype=float32),
 array([948780.1], dtype=float32),
 array([1055883.4], dtype=float32))

In [ ]:
import matplotlib.pyplot as plt
residuals = pred_xg_test - y_test

plt.figure(figsize=(8, 6))
plt.hist(residuals, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='Zero Error')
plt.xlabel('Prediction Error (Residuals)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.legend()
plt.show()
